In [1]:
DICTIONARY_PATH = "./words.txt"

with open(DICTIONARY_PATH, "r") as f:
  words = [word.strip() for word in f.readlines()]

In [2]:
words

['aahed',
 'aalii',
 'aargh',
 'aarti',
 'abaca',
 'abaci',
 'aback',
 'abacs',
 'abaft',
 'abaka',
 'abamp',
 'aband',
 'abase',
 'abash',
 'abask',
 'abate',
 'abaya',
 'abbas',
 'abbed',
 'abbes',
 'abbey',
 'abbot',
 'abcee',
 'abeam',
 'abear',
 'abele',
 'abers',
 'abets',
 'abhor',
 'abide',
 'abies',
 'abled',
 'abler',
 'ables',
 'ablet',
 'ablow',
 'abmho',
 'abode',
 'abohm',
 'aboil',
 'aboma',
 'aboon',
 'abord',
 'abore',
 'abort',
 'about',
 'above',
 'abram',
 'abray',
 'abrim',
 'abrin',
 'abris',
 'absey',
 'absit',
 'abuna',
 'abune',
 'abuse',
 'abuts',
 'abuzz',
 'abyes',
 'abysm',
 'abyss',
 'acais',
 'acari',
 'accas',
 'accoy',
 'acerb',
 'acers',
 'aceta',
 'achar',
 'ached',
 'aches',
 'achoo',
 'acids',
 'acidy',
 'acing',
 'acini',
 'ackee',
 'acker',
 'acmes',
 'acmic',
 'acned',
 'acnes',
 'acock',
 'acold',
 'acorn',
 'acred',
 'acres',
 'acrid',
 'acros',
 'acted',
 'actin',
 'acton',
 'actor',
 'acute',
 'acyls',
 'adage',
 'adapt',
 'adaws',
 'adays',


In [3]:
import string 
print(set(string.ascii_lowercase))


{'o', 'g', 'a', 'f', 'k', 'i', 't', 'x', 'y', 'j', 'b', 'v', 'e', 'c', 'd', 'h', 'm', 'p', 's', 'l', 'n', 'r', 'u', 'q', 'w', 'z'}


In [8]:
import enum 

class WordleGame:
  def __init__(self, solution):
    self.solution = solution
    self.state = {
      "unknown": set(string.ascii_lowercase), # letters that haven't been guessed yet
      "in_word": set(), # letters that are in the word
      "fixed": [None] * solution.length # the word with 
      
    }
    self.state = {c: }

SyntaxError: invalid syntax (2685458465.py, line 12)

In [4]:
import numpy as np

In [6]:
mat = np.zeros((26, 5), np.int32)

for word in words:
  ascii_ixs = [ord(c) - ord('a') for c in word]
  for word_pos, ascii_ix in enumerate(ascii_ixs):
    mat[ascii_ix, word_pos] += 1

mat

array([[ 737, 2263, 1236, 1074,  680],
       [ 909,   81,  335,  243,   59],
       [ 922,  176,  392,  411,  127],
       [ 685,   84,  390,  471,  823],
       [ 303, 1628,  882, 2327, 1522],
       [ 598,   24,  178,  233,   82],
       [ 638,   76,  364,  423,  143],
       [ 489,  546,  120,  235,  370],
       [ 165, 1383, 1051,  880,  280],
       [ 202,   11,   46,   29,    3],
       [ 376,   95,  272,  503,  259],
       [ 577,  699,  848,  771,  476],
       [ 693,  188,  511,  402,  182],
       [ 325,  345,  964,  788,  530],
       [ 262, 2096,  993,  698,  389],
       [ 859,  231,  364,  418,  147],
       [  78,   15,   13,    2,    4],
       [ 628,  940, 1198,  719,  673],
       [1565,   93,  533,  516, 3958],
       [ 815,  239,  616,  898,  727],
       [ 189, 1187,  667,  401,   67],
       [ 242,   52,  240,  156,    4],
       [ 413,  163,  271,  128,   64],
       [  16,   57,  133,   12,   70],
       [ 181,  271,  213,  108, 1301],
       [ 105,   29,  142,

In [15]:
ord("a")

97

In [7]:
"seres" in words

True

In [8]:
len(words)

12972

A strategy is a function f which takes as input a state S which is

- A list of unused letters 
- A list of used letters, and their positions or lack thereof

basically just the words played so far.

It outputs a word W which, when applied to S, will produce a new state S'.

We say S' improves S iff S' contains more information than S, that is – the list of possible words is shorter.

A strategy is a tree with up to 3^5=243 branches per node.

The average depth of the tree is the expected value of the strategy.

Every leaf node is a word.

If a state update filters down to zero words, then the state is impossible and shoulsn't be part of the graph.



In [54]:
from typing import Tuple, Set
from enum import Enum, auto

class LetterPosition(Enum):
  MISSING = auto() # grey
  CORRECT = auto() # green
  WRONG_POSITION = auto() # yellow


def is_possible_word(word: str, letter_data: Set[Tuple[str, int]]):
  for c, position in letter_data:
    if position == 0 and c in word:
      return False 
    if position != 0 and c not in word:
      return False
    if position > 0 and word[position - 1] != c:
      return False
    if position < 0 and word[-position - 1] == c:
      return False
  return True


class State:
  def __init__(self):
    self.words_played = []
    # (c, 0) means character c does not exist in the target string
    # (c, i) means character c exists in the target string at position i (one-indexed)
    # (c, -i) means character c exists in the target string but not at position i (one-indexed)
    self.letter_data: set(Tuple[str, int]) = set()
    self.possible_targets = set(words)
    
    
  # TODO implement copy constructor
  
  def move(self, word: str, outcome: list[LetterPosition]):
    self.words_played.append(word)
    for i, (letter, o) in enumerate(zip(word, outcome)):
      # don't add to the missing list if it's otherwise in the word
      if o == LetterPosition.MISSING and all(l[0] != letter for l in self.letter_data):
        self.letter_data.add((letter, 0))

      elif o == LetterPosition.CORRECT:
        self.letter_data.add((letter, i + 1))
      elif o == LetterPosition.WRONG_POSITION:
        self.letter_data.add((letter, -i - 1))
        
    # todo – this doesn't match exactly; it should be ok to have grey letters if there are repeats
    filtered = {w for w in words if is_possible_word(w, self.letter_data)}
    if len(filtered) == len(self.possible_targets):
      return False # unproductive move; didn't add any info 
    self.possible_targets = filtered
    return True 

In [73]:
s = State()

a = s.move("slope", [LetterPosition.CORRECT, LetterPosition.CORRECT, LetterPosition.MISSING, LetterPosition.WRONG_POSITION, LetterPosition.MISSING])

print(s.possible_targets)

print(s.letter_data)

print(a) 

a = s.move("sloop", [LetterPosition.CORRECT, LetterPosition.CORRECT, LetterPosition.MISSING, LetterPosition.MISSING, LetterPosition.CORRECT])

print(s.possible_targets)

print(s.letter_data)
print(a)


def word_to_mat(word: str):
  mat = np.zeros((26, 5), np.int32)
  ascii_ixs = [ord(c) - ord('a') for c in word]
  for word_pos, ascii_ix in enumerate(ascii_ixs):
    mat[ascii_ix, word_pos] = 1
  return mat
    

def word_and_outcome_to_mats(word: str, outcome: Tuple[LetterPosition, LetterPosition, LetterPosition, LetterPosition, LetterPosition]):
  mat = np.ones((26, 5), np.int32)
  missing_mat = np.zeros((26, 5), np.int32) # this matrix has ones wherever we're sure that a letter cannot be.
  required_mat = np.zeros(26, 5) # this matrix has ones wherever we're sure a letter must be.
  unsure_mat = np.zeros(26, 5) # this matrix has ones across rows where we know letters must exist but we don't know which position. 
  unsure_count = 0 # ANDing the unsure_mat with the word and summing across everything should give at least unsure_count.
  
  ascii_ixs = [ord(c) - ord('a') for c in word]
  for col, (row, o) in enumerate(zip(ascii_ixs, outcome)):
    if o == LetterPosition.CORRECT:
      mat[:, col] = np.zeros(26, np.int32)
      mat[row, col] = 1
    elif o == LetterPosition.WRONG_POSITION:
      mat[row, col] = 0 # how to experess that it's required elsewhere??
      
    # We need list of "required" arrays to express things that we know are there, and a "missing" array to express things that we know are missing. 
    # The required arrays can be ORed together and the sum of the word and that must be the same as the number of required arrays.
    # the missing array should be ANDED with the word and it must have sum 0.
    
    # need to also figure out how to express an exactly-n dep – that is, if we know that there are exactly n instances of some letter (they grey out afterward.)
  return mat
  sum([word_to_mat(a) for a in words])


# next steps:
# For each (word, outcome) pair, precompute the matrices.

{'slump', 'slurp'}
{('p', -4), ('l', 2), ('o', 0), ('s', 1), ('e', 0)}
True
{'slump', 'slurp'}
{('p', -4), ('l', 2), ('p', 5), ('o', 0), ('s', 1), ('e', 0)}
False


array([[ 737, 2263, 1236, 1074,  680],
       [ 909,   81,  335,  243,   59],
       [ 922,  176,  392,  411,  127],
       [ 685,   84,  390,  471,  823],
       [ 303, 1628,  882, 2327, 1522],
       [ 598,   24,  178,  233,   82],
       [ 638,   76,  364,  423,  143],
       [ 489,  546,  120,  235,  370],
       [ 165, 1383, 1051,  880,  280],
       [ 202,   11,   46,   29,    3],
       [ 376,   95,  272,  503,  259],
       [ 577,  699,  848,  771,  476],
       [ 693,  188,  511,  402,  182],
       [ 325,  345,  964,  788,  530],
       [ 262, 2096,  993,  698,  389],
       [ 859,  231,  364,  418,  147],
       [  78,   15,   13,    2,    4],
       [ 628,  940, 1198,  719,  673],
       [1565,   93,  533,  516, 3958],
       [ 815,  239,  616,  898,  727],
       [ 189, 1187,  667,  401,   67],
       [ 242,   52,  240,  156,    4],
       [ 413,  163,  271,  128,   64],
       [  16,   57,  133,   12,   70],
       [ 181,  271,  213,  108, 1301],
       [ 105,   29,  142,

In [63]:
import itertools 

all_outcomes = list(itertools.product([LetterPosition.CORRECT, LetterPosition.MISSING, LetterPosition.WRONG_POSITION], repeat=5))

for word in words:
  print(word)
  for outcome in all_outcomes:
    s = State()
    s.move(word, outcome)


aahed
aalii
aargh
aarti
abaca
abaci
aback
abacs
abaft
abaka
abamp
aband
abase
abash
abask
abate
abaya
abbas
abbed
abbes
abbey
abbot
abcee
abeam
abear
abele
abers
abets
abhor
abide
abies
abled
abler
ables
ablet
ablow
abmho
abode
abohm
aboil
aboma
aboon
abord
abore
abort
about
above
abram
abray
abrim
abrin
abris
absey
absit
abuna
abune
abuse
abuts
abuzz
abyes
abysm
abyss
acais
acari
accas
accoy
acerb
acers
aceta
achar
ached
aches
achoo
acids
acidy
acing
acini
ackee
acker
acmes
acmic
acned
acnes
acock
acold
acorn
acred
acres
acrid
acros
acted
actin
acton
actor
acute
acyls
adage
adapt
adaws
adays
adbot
addax
added
adder
addio
addle
adeem
adept
adhan
adieu
adios
adits
adman
admen
admin
admit
admix
adobe
adobo
adopt
adore
adorn
adown
adoze
adrad
adred
adsum
aduki
adult
adunc
adust
advew
adyta
adzed
adzes
aecia
aedes
aegis
aeons
aerie
aeros
aesir
afald
afara
afars
afear
affix
afire
aflaj
afoot
afore
afoul
afrit
afros
after
again
agama
agami
agape
agars
agast
agate
agave
agaze
agene
agent
ager

KeyboardInterrupt: 

How much harder is hard mode?

Easy to compute if you just limit the subsequent guesses to words that work with hard mode.